In [1]:
import pickle
import scrapbook as sb
import pandas as pd
from dateutil import tz

C:\Users\Sakal\anaconda3\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
df_results = pickle.load(open('df_results.pickle', 'rb'))

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
df_results = df_results.join(df_results['status'].apply(pd.Series))
#df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
#df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))
df_results.pop('status')
df_results.pop('status_type_summary')
df_results['Job Number'] = df_results['properties'].apply(lambda t: t['Job Number'] if 'Job Number' in t else '')
df_results['Lot Number'] = df_results['properties'].apply(lambda t: t['Lot Number'] if 'Lot Number' in t else '')
properties = pd.json_normalize(df_results["properties"])
df_results = pd.concat([df_results.drop(["properties"], axis = 1), properties],axis =1 )

#convert UTC timezone to local timezone
to_zone = tz.tzlocal()
utc = df_results['started_at']
def astimezone(x):
    return x.astimezone(to_zone)
# Convert time zone
central = utc.apply(astimezone)
df_results = pd.concat([df_results.drop(['started_at'],axis=1), central],axis=1)

#convert UTC timezone to local timezone
to_zone = tz.tzlocal()
utc = df_results['updated_at']
def astimezone(x):
    return x.astimezone(to_zone)
# Convert time zone
central = utc.apply(astimezone)
df_results = pd.concat([df_results.drop(['updated_at'],axis=1), central],axis=1)

df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))

In [5]:
df_results_unique = df_results.groupby(['serial_number']).last().reset_index()

In [6]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_results, index=False)['fields'],
    'values': df_results.values.tolist(),
}

results_graph = {
    'type': 'data_frame',
    'id': 'results_graph',
    'data': df_dict
}

result = [results_graph]

In [7]:
sb.glue('result', result)